
# DATABASE SETUP
# WROKING!!!!
![](https://i.imgur.com/nnZkSj8.png)
![](https://i.imgur.com/fSqozNf.png)


# Load dataset

In [1]:
%run ../../src/start.py


python	3.9.6 (default, Jul 30 2021, 16:35:19) 
[GCC 7.5.0]
---------------------
Versions:
----------------------
pandas      1.3.1
numpy       1.20.3
matplotlib  3.4.2
seaborn     0.11.1
plotly      5.1.0
boto3       1.18.16
awswrangler 2.10.0
----------------------


Loaded Libraries
-------------------
import pandas as pd
import numpy as np
import sys,os
import re
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


## AWS
import boto3
import awswrangler as wr
----------------


GLOBAL VARIABLES
--------------------------
# Creating the low level functional client
client = boto3.client(
    's3',
    aws_access_key_id = AWS_KEY_ID,
    aws_secret_access_key = AWS_SECRET,
)
    
# Creating the high level object oriented interface
resource = boto3.resource(
    's3',
    aws_access_key_id = AWS_KEY_I

# load data

In [2]:
# helper function
def bucket_raw_path(bucket_name,path_dir):
    ''''''
    raw_path = f's3://{bucket_name}/{path_dir}'
    return raw_path

In [3]:
raw = bucket_raw_path('dend-data',f'capstone/load-data/')
raw

's3://dend-data/capstone/load-data/'

In [4]:
file = wr.s3.list_objects(raw)[1]
file

's3://dend-data/capstone/load-data/crime-weather-sample-100-09-18.csv'

In [5]:
df = wr.s3.read_csv(file)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            100 non-null    object 
 1   offenses             100 non-null    int64  
 2   offense_type         100 non-null    object 
 3   block_range          100 non-null    object 
 4   street_name          100 non-null    object 
 5   type                 100 non-null    object 
 6   suffix               100 non-null    object 
 7   beat                 100 non-null    object 
 8   premise              100 non-null    object 
 9   date                 100 non-null    object 
 10  hour                 100 non-null    int64  
 11  year                 100 non-null    int64  
 12  premise_description  100 non-null    object 
 13  temp                 100 non-null    float64
 14  feels_like           100 non-null    float64
 15  temp_min             100 non-null    floa

In [7]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,19,2018,restaurant or cafeteria parking lot,82.292,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,10,2009,residence house,63.158,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,17,2016,apartment,66.380,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,17,2013,other unknown,76.172,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,0,2018,apartment parking lot,68.432,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds


# TEST IMPORT

In [8]:

import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

# Police Beat

In [9]:
record = df[['beat']].values[0].tolist()
record

['1A50']

## insert record to policebeat table

In [10]:
# conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
# cur = conn.cursor()



conn = psycopg2.connect(database="crimeweatherdb",
user="frank",
password="punkin99",
host="database-1.cx3dxtsihxox.us-west-2.rds.amazonaws.com",
port='5432')

cur = conn.cursor()

# Load all police beats

In [11]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,19,2018,restaurant or cafeteria parking lot,82.292,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,10,2009,residence house,63.158,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,17,2016,apartment,66.380,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,17,2013,other unknown,76.172,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,0,2018,apartment parking lot,68.432,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds


## OFFENSE_TABLE

In [12]:
offense_list = df.offense_type.unique().tolist()
offense_list

['Theft', 'Burglary', 'Robbery', 'Aggravated Assault', 'Auto Theft']

In [13]:
df_offense = pd.DataFrame(offense_list,columns=['offense_name'])
df_offense

,offense_name
0,Theft
1,Burglary
2,Robbery
3,Aggravated Assault
4,Auto Theft


In [14]:
df_offense = df_offense.reset_index()
df_offense.rename(columns={'index': 'offense_pk'}, inplace=True)
df_offense.offense_pk = df_offense.offense_pk +1
df_offense

,offense_pk,offense_name
0,1,Theft
1,2,Burglary
2,3,Robbery
3,4,Aggravated Assault
4,5,Auto Theft


## BEAT_TABLE

In [15]:
beat_list = df.beat.unique().tolist()
df_beat = pd.DataFrame(beat_list,columns=['beat_name'])

df_beat = df_beat.reset_index()
df_beat.rename(columns={'index': 'beat_pk'}, inplace=True)
df_beat.beat_pk = df_beat.beat_pk +1
df_beat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   beat_pk    57 non-null     int64 
 1   beat_name  57 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.0+ KB


In [16]:
df_beat.head()

,beat_pk,beat_name
0,1,1A50
1,2,15E10
2,3,17E40
3,4,9C20
4,5,16E30


In [17]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,19,2018,restaurant or cafeteria parking lot,82.292,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,10,2009,residence house,63.158,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,17,2016,apartment,66.380,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,17,2013,other unknown,76.172,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,0,2018,apartment parking lot,68.432,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds


## PREMISE_DESCRIPTION_TABLE

In [18]:
premise_list = df.premise_description.unique().tolist()
df_premise = pd.DataFrame(premise_list,columns=['premise_name'])

df_premise = df_premise.reset_index()
df_premise.rename(columns={'index': 'premise_pk'}, inplace=True)
df_premise.premise_pk = df_premise.premise_pk +1
df_premise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   premise_pk    40 non-null     int64 
 1   premise_name  40 non-null     object
dtypes: int64(1), object(1)
memory usage: 768.0+ bytes


In [19]:
df_premise.head()

,premise_pk,premise_name
0,1,restaurant or cafeteria parking lot
1,2,residence house
2,3,apartment
3,4,other unknown
4,5,apartment parking lot


In [20]:
df_premise.head()

,premise_pk,premise_name
0,1,restaurant or cafeteria parking lot
1,2,residence house
2,3,apartment
3,4,other unknown
4,5,apartment parking lot


## Address_TABLE

In [21]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,19,2018,restaurant or cafeteria parking lot,82.292,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,10,2009,residence house,63.158,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,17,2016,apartment,66.380,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,17,2013,other unknown,76.172,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,0,2018,apartment parking lot,68.432,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds


In [22]:
df.block_range.unique()

array(['5100-5199', '5400-5499', '6300-6399', '7900-7999', '8900-8999',
       '14000-14099', '5900-5999', '1300-1399', '7100-7199', '8800-8899',
       '1500-1599', '8100-8199', '12200-12299', '9500-9599',
       '10800-10899', '2400-2499', '11200-11299', '1900-1999',
       '6700-6799', '800-899', '2100-2199', '10500-10599', '13300-13399',
       '1400-1499', 'UNK', '3600-3699', '5000-5099', '6400-6499',
       '7300-7399', '7000-7099', '2000-2099', '4100-4199', '10900-10999',
       '100-199', '14700-14799', '6900-6999', '8400-8499', '3400-3499',
       '7800-7899', '10400-10499', '6200-6299', '12800-12899',
       '11300-11399', '9800-9899', '900-999', '1700-1799', '17700-17799',
       '9600-9699', '3800-3899', '8600-8699', '500-599', '7500-7599',
       '700-799', '7700-7799', '7400-7499', '14400-14499', '3200-3299',
       '200-299', '10300-10399', '12600-12699', '9100-9199', '3100-3199',
       '1100-1199', '2300-2399', '5300-5399', '9700-9799', '8500-8599',
       '10100-10199

In [23]:
df.block_range = df.block_range.str.strip()

In [24]:
df.block_range.replace('UNK','10-100',inplace=True)

In [25]:
df.block_range.unique()

array(['5100-5199', '5400-5499', '6300-6399', '7900-7999', '8900-8999',
       '14000-14099', '5900-5999', '1300-1399', '7100-7199', '8800-8899',
       '1500-1599', '8100-8199', '12200-12299', '9500-9599',
       '10800-10899', '2400-2499', '11200-11299', '1900-1999',
       '6700-6799', '800-899', '2100-2199', '10500-10599', '13300-13399',
       '1400-1499', '10-100', '3600-3699', '5000-5099', '6400-6499',
       '7300-7399', '7000-7099', '2000-2099', '4100-4199', '10900-10999',
       '100-199', '14700-14799', '6900-6999', '8400-8499', '3400-3499',
       '7800-7899', '10400-10499', '6200-6299', '12800-12899',
       '11300-11399', '9800-9899', '900-999', '1700-1799', '17700-17799',
       '9600-9699', '3800-3899', '8600-8699', '500-599', '7500-7599',
       '700-799', '7700-7799', '7400-7499', '14400-14499', '3200-3299',
       '200-299', '10300-10399', '12600-12699', '9100-9199', '3100-3199',
       '1100-1199', '2300-2399', '5300-5399', '9700-9799', '8500-8599',
       '10100-10

In [26]:
def block_range_split(df):
    '''split blockrange col values
    then give median value as a string'''
    first = df.block_range.str.split(pat='-',expand=True)[0].astype('int')
    second = df.block_range.str.split(pat='-',expand=True)[1].astype('int')
    med = np.ceil((second + first)/2).astype('int')
    med = med.astype('str')
    street = df.street_name
    res = med + " " + street
    return res

In [27]:
df['full_address'] = block_range_split(df)

In [28]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description,full_address
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,19,2018,restaurant or cafeteria parking lot,82.292,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain,5150 buffalo speedway
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,10,2009,residence house,63.158,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear,5450 lymbar
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,17,2016,apartment,66.380,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear,6350 bellfort
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,17,2013,other unknown,76.172,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear,7950 wallisville
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,0,2018,apartment parking lot,68.432,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds,8950 braesmont


## FULL ADDRESS

In [29]:
add_list = df.full_address.unique().tolist()
df_address = pd.DataFrame(add_list,columns=['full_address'])

df_address = df_address.reset_index()
df_address.rename(columns={'index': 'address_pk'}, inplace=True)
df_address.address_pk = df_address.address_pk +1
df_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address_pk    98 non-null     int64 
 1   full_address  98 non-null     object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [30]:
df_address.head()

,address_pk,full_address
0,1,5150 buffalo speedway
1,2,5450 lymbar
2,3,6350 bellfort
3,4,7950 wallisville
4,5,8950 braesmont


# DATE_TIME_TABLE

In [31]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description,full_address
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,19,2018,restaurant or cafeteria parking lot,82.292,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain,5150 buffalo speedway
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,10,2009,residence house,63.158,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear,5450 lymbar
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,17,2016,apartment,66.380,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear,6350 bellfort
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,17,2013,other unknown,76.172,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear,7950 wallisville
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,0,2018,apartment parking lot,68.432,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds,8950 braesmont


In [32]:
time_list = df.date_time.unique().tolist()

df_time = pd.DataFrame(time_list,columns=['date_time'])


df_time = df_time.reset_index()
df_time.rename(columns={'index': 'date_time_pk'}, inplace=True)
df_time.date_time_pk = df_time.date_time_pk +1
df_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date_time_pk  100 non-null    int64 
 1   date_time     100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [33]:
df_time.head()

,date_time_pk,date_time
0,1,2018-04-13 19:00:00
1,2,2009-11-10 10:00:00
2,3,2016-02-27 17:00:00
3,4,2013-10-26 17:00:00
4,5,2018-03-10 00:00:00


In [34]:
# convert timestamp column to datetime
t = pd.to_datetime(df_time['date_time'])
# insert time data records
time_data = (df_time.date_time_pk,df_time.date_time,t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year,t.dt.weekday)
column_labels = ('date_time_pk','date_time', 'hour', 'day','week','month','year','weekday')
date_time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
date_time_df.head()

<ipython-input-34-dd57859b4dca>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,date_time_pk,date_time,hour,day,week,month,year,weekday
0,1,2018-04-13 19:00:00,19,13,15,4,2018,4
1,2,2009-11-10 10:00:00,10,10,46,11,2009,1
2,3,2016-02-27 17:00:00,17,27,8,2,2016,5
3,4,2013-10-26 17:00:00,17,26,43,10,2013,5
4,5,2018-03-10 00:00:00,0,10,10,3,2018,5


In [35]:
date_time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date_time_pk  100 non-null    int64 
 1   date_time     100 non-null    object
 2   hour          100 non-null    int64 
 3   day           100 non-null    int64 
 4   week          100 non-null    int64 
 5   month         100 non-null    int64 
 6   year          100 non-null    int64 
 7   weekday       100 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 6.4+ KB


# Crime_Fact_Table

In [36]:
df.head()

,date_time,offenses,offense_type,block_range,street_name,type,suffix,beat,premise,date,hour,year,premise_description,temp,feels_like,temp_min,temp_max,humidity_per,wind_speed,rain_vol_1h_mm,snow_vol_1h_mm,clouds_all_per,weather_main,weather_description,full_address
0,2018-04-13 19:00:00,1,Theft,5100-5199,buffalo speedway,-,-,1A50,Restaurant or Cafeteria Parking Lot,04/13/2018,19,2018,restaurant or cafeteria parking lot,82.292,84.650,80.654,83.102,59,7.7,0.56,0.0,75,Rain,light rain,5150 buffalo speedway
1,2009-11-10 10:00:00,1,Burglary,5400-5499,lymbar,dr,-,15E10,20R,11/10/09 00:00:00,10,2009,residence house,63.158,63.500,62.654,64.418,92,4.1,0.00,0.0,1,Clear,sky is clear,5450 lymbar
2,2016-02-27 17:00:00,1,Burglary,6300-6399,bellfort,st,W,17E40,Apartment,2016-02-27 00:00:00,17,2016,apartment,66.380,64.184,64.094,66.902,31,2.1,0.00,0.0,1,Clear,sky is clear,6350 bellfort
3,2013-10-26 17:00:00,1,Theft,7900-7999,wallisville,rd,-,9C20,250,2013-10-26 00:00:00,17,2013,other unknown,76.172,76.406,73.616,78.818,62,5.7,0.00,0.0,1,Clear,sky is clear,7950 wallisville
4,2018-03-10 00:00:00,1,Robbery,8900-8999,braesmont,dr,-,15E10,Apartment Parking Lot,03/10/2018,0,2018,apartment parking lot,68.432,68.036,68.000,70.016,65,4.1,0.00,0.0,90,Clouds,overcast clouds,8950 braesmont


In [37]:
df = df[['date_time',
 'offenses',
 'offense_type',
 'beat',
 'premise_description',
 'temp',
 'feels_like',
 'humidity_per',
 'rain_vol_1h_mm',
 'snow_vol_1h_mm',
 'full_address']]

In [38]:
df.columns.tolist()

['date_time',
 'offenses',
 'offense_type',
 'beat',
 'premise_description',
 'temp',
 'feels_like',
 'humidity_per',
 'rain_vol_1h_mm',
 'snow_vol_1h_mm',
 'full_address']

In [39]:
crime_fact = df.copy()
crime_fact.head()

,date_time,offenses,offense_type,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,full_address
0,2018-04-13 19:00:00,1,Theft,1A50,restaurant or cafeteria parking lot,82.292,84.650,59,0.56,0.0,5150 buffalo speedway
1,2009-11-10 10:00:00,1,Burglary,15E10,residence house,63.158,63.500,92,0.00,0.0,5450 lymbar
2,2016-02-27 17:00:00,1,Burglary,17E40,apartment,66.380,64.184,31,0.00,0.0,6350 bellfort
3,2013-10-26 17:00:00,1,Theft,9C20,other unknown,76.172,76.406,62,0.00,0.0,7950 wallisville
4,2018-03-10 00:00:00,1,Robbery,15E10,apartment parking lot,68.432,68.036,65,0.00,0.0,8950 braesmont


In [40]:
crime_fact.head()

,date_time,offenses,offense_type,beat,premise_description,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,full_address
0,2018-04-13 19:00:00,1,Theft,1A50,restaurant or cafeteria parking lot,82.292,84.650,59,0.56,0.0,5150 buffalo speedway
1,2009-11-10 10:00:00,1,Burglary,15E10,residence house,63.158,63.500,92,0.00,0.0,5450 lymbar
2,2016-02-27 17:00:00,1,Burglary,17E40,apartment,66.380,64.184,31,0.00,0.0,6350 bellfort
3,2013-10-26 17:00:00,1,Theft,9C20,other unknown,76.172,76.406,62,0.00,0.0,7950 wallisville
4,2018-03-10 00:00:00,1,Robbery,15E10,apartment parking lot,68.432,68.036,65,0.00,0.0,8950 braesmont


# PREMISE_SPLIT

In [41]:
crime_fact = crime_fact.merge(df_premise, left_on='premise_description', right_on='premise_name', suffixes=(True,True))
crime_fact.drop(['premise_description', 'premise_name'], axis=1,inplace=True)

In [42]:
crime_fact.head()

,date_time,offenses,offense_type,beat,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,full_address,premise_pk
0,2018-04-13 19:00:00,1,Theft,1A50,82.292,84.650,59,0.56,0.0,5150 buffalo speedway,1
1,2015-08-21 22:00:00,1,Theft,1A30,90.914,97.358,52,0.00,0.0,2450 richmond,1
2,2009-11-10 10:00:00,1,Burglary,15E10,63.158,63.500,92,0.00,0.0,5450 lymbar,2
3,2014-01-03 14:00:00,1,Burglary,20G40,34.682,30.578,66,0.00,0.0,1350 briar bayou,2
4,2009-11-28 20:00:00,1,Burglary,12D10,68.936,68.630,66,0.00,0.0,10850 coral garden,2


# ADDRESS_SPLIT

In [43]:
crime_fact = crime_fact.merge(df_address, left_on='full_address', right_on='full_address', suffixes=(True,True))
crime_fact.drop(['full_address'], axis=1,inplace=True)

In [44]:
crime_fact.head()

,date_time,offenses,offense_type,beat,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,premise_pk,address_pk
0,2018-04-13 19:00:00,1,Theft,1A50,82.292,84.650,59,0.56,0.0,1,1
1,2015-08-21 22:00:00,1,Theft,1A30,90.914,97.358,52,0.00,0.0,1,98
2,2009-11-10 10:00:00,1,Burglary,15E10,63.158,63.500,92,0.00,0.0,2,2
3,2014-01-03 14:00:00,1,Burglary,20G40,34.682,30.578,66,0.00,0.0,2,8
4,2009-11-28 20:00:00,1,Burglary,12D10,68.936,68.630,66,0.00,0.0,2,15


# BEAT SPLIT

In [45]:
crime_fact =crime_fact.merge(df_beat, left_on='beat', right_on='beat_name', suffixes=(True,True))
crime_fact.drop(['beat_name','beat'], axis=1,inplace=True)

In [46]:
crime_fact.head()

,date_time,offenses,offense_type,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,premise_pk,address_pk,beat_pk
0,2018-04-13 19:00:00,1,Theft,82.292,84.650,59,0.56,0.0,1,1,1
1,2014-07-02 07:00:00,1,Burglary,79.124,79.124,85,0.00,0.0,21,31,1
2,2016-08-11 17:00:00,1,Theft,93.434,106.034,58,0.10,0.0,29,63,1
3,2013-09-13 17:00:00,1,Theft,91.544,94.712,44,0.00,0.0,33,69,1
4,2015-08-21 22:00:00,1,Theft,90.914,97.358,52,0.00,0.0,1,98,57


# OFFENSE_SPLIT

In [47]:
crime_fact = crime_fact.merge(df_offense, left_on='offense_type', right_on='offense_name', suffixes=(True,True))
crime_fact.drop(['offense_type','offense_name'], axis=1,inplace=True)

In [48]:
crime_fact.head()

,date_time,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,premise_pk,address_pk,beat_pk,offense_pk
0,2018-04-13 19:00:00,1,82.292,84.650,59,0.56,0.0,1,1,1,1
1,2016-08-11 17:00:00,1,93.434,106.034,58,0.10,0.0,29,63,1,1
2,2013-09-13 17:00:00,1,91.544,94.712,44,0.00,0.0,33,69,1,1
3,2015-08-21 22:00:00,1,90.914,97.358,52,0.00,0.0,1,98,57,1
4,2012-02-08 18:00:00,1,58.226,56.282,54,0.00,0.0,5,10,2,1


# TIME SPLIT

In [49]:
crime_fact.head()

,date_time,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,premise_pk,address_pk,beat_pk,offense_pk
0,2018-04-13 19:00:00,1,82.292,84.650,59,0.56,0.0,1,1,1,1
1,2016-08-11 17:00:00,1,93.434,106.034,58,0.10,0.0,29,63,1,1
2,2013-09-13 17:00:00,1,91.544,94.712,44,0.00,0.0,33,69,1,1
3,2015-08-21 22:00:00,1,90.914,97.358,52,0.00,0.0,1,98,57,1
4,2012-02-08 18:00:00,1,58.226,56.282,54,0.00,0.0,5,10,2,1


In [50]:
crime_fact = crime_fact.merge(date_time_df, left_on='date_time', right_on='date_time', suffixes=(True,True))
crime_fact.drop(['date_time','hour','day','week','month','year','weekday'], axis=1,inplace=True)

In [51]:
crime_fact.head()

,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,premise_pk,address_pk,beat_pk,offense_pk,date_time_pk
0,1,82.292,84.650,59,0.56,0.0,1,1,1,1,1
1,1,93.434,106.034,58,0.10,0.0,29,63,1,1,64
2,1,91.544,94.712,44,0.00,0.0,33,69,1,1,70
3,1,90.914,97.358,52,0.00,0.0,1,98,57,1,100
4,1,58.226,56.282,54,0.00,0.0,5,10,2,1,10


In [52]:
crime_fact.head()

,offenses,temp,feels_like,humidity_per,rain_vol_1h_mm,snow_vol_1h_mm,premise_pk,address_pk,beat_pk,offense_pk,date_time_pk
0,1,82.292,84.650,59,0.56,0.0,1,1,1,1,1
1,1,93.434,106.034,58,0.10,0.0,29,63,1,1,64
2,1,91.544,94.712,44,0.00,0.0,33,69,1,1,70
3,1,90.914,97.358,52,0.00,0.0,1,98,57,1,100
4,1,58.226,56.282,54,0.00,0.0,5,10,2,1,10


In [53]:
crime_fact.rename(columns={'offenses': 'numoffenses', 'rain_vol_1h_mm': 'rain','snow_vol_1h_mm':'snow','humidity_per':'humidity'}, inplace=True)

In [54]:
crime_fact.head()

,numoffenses,temp,feels_like,humidity,rain,snow,premise_pk,address_pk,beat_pk,offense_pk,date_time_pk
0,1,82.292,84.650,59,0.56,0.0,1,1,1,1,1
1,1,93.434,106.034,58,0.10,0.0,29,63,1,1,64
2,1,91.544,94.712,44,0.00,0.0,33,69,1,1,70
3,1,90.914,97.358,52,0.00,0.0,1,98,57,1,100
4,1,58.226,56.282,54,0.00,0.0,5,10,2,1,10


In [55]:
crime_fact = crime_fact.reset_index()
crime_fact.rename(columns={'index': 'pk'}, inplace=True)
crime_fact.pk = crime_fact.pk +1
crime_fact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pk            100 non-null    int64  
 1   numoffenses   100 non-null    int64  
 2   temp          100 non-null    float64
 3   feels_like    100 non-null    float64
 4   humidity      100 non-null    int64  
 5   rain          100 non-null    float64
 6   snow          100 non-null    float64
 7   premise_pk    100 non-null    int64  
 8   address_pk    100 non-null    int64  
 9   beat_pk       100 non-null    int64  
 10  offense_pk    100 non-null    int64  
 11  date_time_pk  100 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 9.5 KB


In [56]:
crime_fact.head()

,pk,numoffenses,temp,feels_like,humidity,rain,snow,premise_pk,address_pk,beat_pk,offense_pk,date_time_pk
0,1,1,82.292,84.650,59,0.56,0.0,1,1,1,1,1
1,2,1,93.434,106.034,58,0.10,0.0,29,63,1,1,64
2,3,1,91.544,94.712,44,0.00,0.0,33,69,1,1,70
3,4,1,90.914,97.358,52,0.00,0.0,1,98,57,1,100
4,5,1,58.226,56.282,54,0.00,0.0,5,10,2,1,10


In [57]:
crime_fact.columns.tolist()

['pk',
 'numoffenses',
 'temp',
 'feels_like',
 'humidity',
 'rain',
 'snow',
 'premise_pk',
 'address_pk',
 'beat_pk',
 'offense_pk',
 'date_time_pk']

In [58]:
crime_fact = crime_fact[[ 'pk', 'numoffenses','temp', 'humidity', 'feels_like',
 'rain',
 'snow',
'date_time_pk',
 'address_pk',
 'premise_pk',
'offense_pk',
 'beat_pk']]

In [59]:
crime_fact.head()

,pk,numoffenses,temp,humidity,feels_like,rain,snow,date_time_pk,address_pk,premise_pk,offense_pk,beat_pk
0,1,1,82.292,59,84.650,0.56,0.0,1,1,1,1,1
1,2,1,93.434,58,106.034,0.10,0.0,64,63,29,1,1
2,3,1,91.544,44,94.712,0.00,0.0,70,69,33,1,1
3,4,1,90.914,52,97.358,0.00,0.0,100,98,1,1,57
4,5,1,58.226,54,56.282,0.00,0.0,10,10,5,1,2


# UPLOAD DTA

In [60]:

conn = psycopg2.connect(database="crimeweatherdb",
user="frank",
password="punkin99",
host="database-1.cx3dxtsihxox.us-west-2.rds.amazonaws.com",
port='5432')

cur = conn.cursor()

In [61]:
df_premise.head()

,premise_pk,premise_name
0,1,restaurant or cafeteria parking lot
1,2,residence house
2,3,apartment
3,4,other unknown
4,5,apartment parking lot


# premise upload

In [62]:
for i in df_premise.values.tolist():
    cur.execute(premise_table_insert, i)
    conn.commit()

# Address upload

In [63]:
df_address.head()

,address_pk,full_address
0,1,5150 buffalo speedway
1,2,5450 lymbar
2,3,6350 bellfort
3,4,7950 wallisville
4,5,8950 braesmont


In [64]:
for i in df_address.values.tolist():
    cur.execute(address_table_insert, i)
    conn.commit()

# df_beat

In [65]:
df_beat.head()

,beat_pk,beat_name
0,1,1A50
1,2,15E10
2,3,17E40
3,4,9C20
4,5,16E30


In [66]:
for i in df_beat.values.tolist():
    cur.execute(police_beat_table_insert, i)
    conn.commit()

# df_offense load

In [67]:
df_offense

,offense_pk,offense_name
0,1,Theft
1,2,Burglary
2,3,Robbery
3,4,Aggravated Assault
4,5,Auto Theft


In [68]:
for i in df_offense.values.tolist():
    cur.execute(offense_table_insert, i)
    conn.commit()

## date_time_df

In [69]:
date_time_df.head()

,date_time_pk,date_time,hour,day,week,month,year,weekday
0,1,2018-04-13 19:00:00,19,13,15,4,2018,4
1,2,2009-11-10 10:00:00,10,10,46,11,2009,1
2,3,2016-02-27 17:00:00,17,27,8,2,2016,5
3,4,2013-10-26 17:00:00,17,26,43,10,2013,5
4,5,2018-03-10 00:00:00,0,10,10,3,2018,5


In [70]:
for i in date_time_df.values.tolist():
    cur.execute(datetime_table_insert, i)
    conn.commit()

In [71]:
datetime_table_insert

'\nINSERT INTO datetime_dim (\n\tdatetime_id,\n\tdate_time,\n    hour,\n    day,\n    week ,\n    month ,\n    year,\n    weekday\n)\nVALUES (%s,%s, %s, %s, %s, %s, %s, %s)\nON CONFLICT (datetime_id) DO NOTHING\n'

# crime fact upload

In [72]:
crime_fact.head()

,pk,numoffenses,temp,humidity,feels_like,rain,snow,date_time_pk,address_pk,premise_pk,offense_pk,beat_pk
0,1,1,82.292,59,84.650,0.56,0.0,1,1,1,1,1
1,2,1,93.434,58,106.034,0.10,0.0,64,63,29,1,1
2,3,1,91.544,44,94.712,0.00,0.0,70,69,33,1,1
3,4,1,90.914,52,97.358,0.00,0.0,100,98,1,1,57
4,5,1,58.226,54,56.282,0.00,0.0,10,10,5,1,2


In [73]:
for i in crime_fact.values.tolist():
    cur.execute(crime_fact_table_insert, i)
    conn.commit()